In [ ]:
from binance.client import Client
import pandas as pd
import json
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
from security import get_keys
binance_key = get_keys('client')
client = Client(binance_key['api_key'], binance_key['api_secret'])

In [ ]:
all_symbol_ticker = pd.DataFrame(client.get_symbol_ticker())
all_balances = pd.DataFrame(client.get_account()['balances'])

In [93]:
def printAsset():
    BTC = client.get_asset_balance('BTC', recvWindow=10000)['free']
    USDT = client.get_asset_balance('USDT', recvWindow=10000)['free']
    print('BTC', BTC)
    print('USDT', USDT)
    print('PHP', float(USDT)*50.0)

In [96]:
printAsset()

BTC 0.00000000
USDT 46.68062465
PHP 2334.0312325


In [ ]:
symbols = ['USDT', 'BTC', 'ETH', 'XRP']
expected_pairs = []
for sym1 in symbols:
    for sym2 in symbols:
        if sym1 == sym2:
            continue
        expected_pairs.append(sym1 + sym2)
pairs = [x for x in all_symbol_ticker['symbol'] if x in expected_pairs]
pairs

In [ ]:
def show_average(min_num):
    for pair in pairs:
        if 'USDT' in pair:
            data = client.get_historical_klines(pair, client.KLINE_INTERVAL_1MINUTE, str(min_num) + " min ago UTC")
            disp = pd.DataFrame(data, columns = ['Open time',
                                                'Open',
                                                'High',
                                                'Low',
                                                'Close',
                                                'Volume',
                                                'Close time',
                                                'Quote asset volume',
                                                'Number of trades',
                                                'Taker buy base asset volume',
                                                'Taker buy quote asset volume',
                                                'Can be ignored',])
            y = np.array(disp['Close'].astype(float))
            # x = np.array(disp['Close time'].astype(float)).reshape((-1, 1))
            x = np.linspace(0.0, float(min_num) - 1.0, num = min_num).reshape((-1, 1))
            slope = LinearRegression().fit(x, y).coef_
            print(pair[:len('USDT')-1], slope[0], 'USDT/min')

In [ ]:
def buyAmount(coin, pair):
    balanceBuy = float(client.get_asset_balance(coin,
    recvWindow=10000)['free'])
    close = float(client.get_symbol_ticker(symbol=pair)['price'])
    maxBuy = round(balanceBuy / close * .995, 5)
    return maxBuy
    
def sellAmount(coin):
    balanceSell = float(client.get_asset_balance(coin,
    recvWindow=10000)['free'])
    maxSell = round(balanceSell * .995, 5)
    return maxSell

In [121]:
buyAmount('USDT', 'BTCUSDT')

0.00434

In [ ]:
sellAmount('BTC')

In [ ]:
def buy(amount, pair):
    client.create_test_order(
    symbol=pair,
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=amount,
    recvWindow=10000)
    print('Buy: {}'.format(amount))
    
def sell(amount, pair):
    client.create_test_order(
    symbol=pair,
    side=Client.SIDE_SELL,
    type=Client.ORDER_TYPE_MARKET,
    quantity=amount,
    recvWindow=10000)
    print('Sell: {}'.format(amount))

In [ ]:
def buyBTC():
    amount = buyAmount('USDT', 'BTCUSDT')
    buy(amount, 'BTCUSDT')

def sellBTC():
    amount = sellAmount('USDT')
    sell(amount, 'BTCUSDT')

In [108]:
# maxBuy = buyAmount('USDT', 'BTCUSDT')
# buy(maxBuy, 'BTCUSDT')
# maxSell = sellAmount('USDT')
# sell(maxSell, 'BTCUSDT')
buyBTC()

Buy: 0.00434


In [109]:
sellBTC()

Sell: 46.44722


In [106]:
show_average(3)
show_average(10)

BTC -2.349999999999453 USDT/min
ETH -0.09499999999999886 USDT/min
XRP -1.5000000000001125e-05 USDT/min
BTC 0.302545454545499 USDT/min
ETH 0.048242424242426876 USDT/min
XRP -1.539393939393898e-05 USDT/min


In [119]:
printAsset()

BTC 0.00000000
USDT 46.68062465
PHP 2334.0312325
